### Import libraries



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional, Embedding, Flatten
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from langdetect import detect
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam



2023-12-18 21:03:13.768992: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-18 21:03:13.807958: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 21:03:13.807997: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 21:03:13.809347: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-18 21:03:13.815782: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-18 21:03:13.816578: I tensorflow/core/platform/cpu_feature_guard.cc:1

### Dataset

In [2]:
dataset = pd.read_csv('preprocessed_train.csv')

# give a description of the data
print(dataset.describe())

         Unnamed: 0        rating  preprocessed_review_length
count  31246.000000  31246.000000                31246.000000
mean   15907.414485      0.238174                   41.354733
std     9259.086027      0.946370                   51.817764
min        0.000000     -1.000000                    0.000000
25%     7863.250000     -1.000000                   13.000000
50%    15854.500000      1.000000                   25.000000
75%    23924.750000      1.000000                   50.000000
max    32035.000000      1.000000                 1528.000000


In [3]:

df = pd.read_csv('preprocessed_train.csv')


# Drop rows with NaN values in the 'preprocessed_review' column
df = df.dropna(subset=['preprocessed_review'])

# Convert ratings to one-hot encoded labels
labels = to_categorical(df['rating'] + 1)  # Adding 1 to convert -1, 0, 1 to 0, 1, 2

# Tokenize the Arabic text
tokenizer_arabic = Tokenizer()
tokenizer_arabic.fit_on_texts(df['preprocessed_review'])
sequences_arabic = tokenizer_arabic.texts_to_sequences(df['preprocessed_review'])
padded_sequences_arabic = pad_sequences(sequences_arabic)

# Define the LSTM model with dropout layers
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer_arabic.word_index) + 1, output_dim=100, input_length=padded_sequences_arabic.shape[1]))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2)) # Adding dropout to the LSTM layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))  # Adding dropout to the Dense layer
model.add(Dense(3, activation='softmax'))  # Three output nodes for negative, neutral, and positive

# Compile the model with categorical_crossentropy
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on Arabic data
model.fit(padded_sequences_arabic, labels, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
773/773 [==============================] - 100s 128ms/step - loss: 0.5235 - accuracy: 0.8020 - val_loss: 0.4551 - val_accuracy: 0.8307
Epoch 2/5
773/773 [==============================] - 94s 122ms/step - loss: 0.3509 - accuracy: 0.8763 - val_loss: 0.4670 - val_accuracy: 0.8314
Epoch 3/5
773/773 [==============================] - 96s 124ms/step - loss: 0.3088 - accuracy: 0.8968 - val_loss: 0.5158 - val_accuracy: 0.8223
Epoch 4/5
773/773 [==============================] - 95s 123ms/step - loss: 0.2151 - accuracy: 0.9288 - val_loss: 0.5717 - val_accuracy: 0.8209
Epoch 5/5
773/773 [==============================] - 94s 122ms/step - loss: 0.1752 - accuracy: 0.9425 - val_loss: 0.6397 - val_accuracy: 0.8112


In [4]:
# show the accuracy of the model
loss, accuracy = model.evaluate(padded_sequences_arabic, labels, verbose=False)

print("Training Accuracy: {:.4f}".format(accuracy))




Training Accuracy: 0.9284


In [ ]:
# now we will test the model on the test data
# The submission file should be a .csv file that contains two columns: ID, and rating. The ID is the id of the Arabic sentences and the rating is the predicted sentiment analysis and should be one of the following values: 1,0,-1

# read the test data
test_data = pd.read_csv('test.csv')

# preprocess the test data
